## Group 05: Abstract Cars4You - exactly 300 words

The Cars4You project aims to accelerate and standardize used-car price evaluations by replacing manual, subjective pricing with a production-ready machine learning pipeline. Our objective was to maximize predictive accuracy (MAE) on unseen cars while ensuring robustness to wrong inputs and a leakage-free evaluation.

Our EDA containing univariate, bivariate and multivariate analysis showed three dominant challenges: (1) inconsistent raw entries (typos, invalid ranges, sparse categories), (2) strong segmentation effects by brand/model, and (3) heavy-tailed numeric distributions (notably mileage). 

We addressed these with a custom engineered and reproducible sklearn pipeline. It follows the state of the art pipeline archtecture and consists the following transformers: 

deterministic cleaning and category canonicalization with `CarDataCleaner`, leakage-safe outlier handling via winsorization with `OutlierHandler`, and hierarchical, segment-aware imputation with `GroupImputer`. We then added domain-informed feature engineering with `CarFeatureEngineer` to encode depreciation, usage intensity, efficiency/performance ratios, interaction effects, and relative positioning within brand/model segments.

Encoding and scaling were consolidated in a `ColumnTransformer` combining selective log transforms, `RobustScaler`, one-hot encoding, and median target encoding for high-signal categorical structure.

To reduce noise and improve generalization, we implemented automated feature selection as a dedicated pipeline stage: VarianceThreshold followed by majority voting across complementary selectors (Spearman relevance+redundancy, mutual information, and tree-based importance). SHAP was used strictly for interpretability and diagnostics in the end.

All model selection and tuning followed a consistent 5-fold cross-validation protocol. Primary evaluation metric MAE was set at the beginning of the project, we also evaluated RMSE and R2. After a first run of different models on original and engineered features, further hyperparameter tuning on the tree-based models HistGradientBoost and RandomForest was decided. The final tuned RF pipeline improved substantially over a naive median baseline (MAE ≈ 6.8k), achieving approximately **£1.3k MAE** in cross-validation.

For detailed methodological reasoning, trade-offs and further findings please refer to the respective sections.

Data Cleaning Section:

**Our approach:**
- We `clean data inconsistencies` and data entry errors that we found in the EDA
- These columns will be `set to NaN` for that specific entry to not lose rows in the data due to removing
- Afterwards, this value will be imputed (see Section 2.3)

____

**Numerical Features**

| **Feature** | **Allowed thresholds** | **Reasoning** | **# filtered below threshold** | **# filtered above threshold** |
| :--- | :--- | :--- | :---: | :---: |
| `year` | 1886 to 2020 | The first automobile is dated to 1886 (Benz Patent Motor Car), so earlier values are implausible; the dataset is from 2020, so newer model years are logically impossible. [1] | 0 | 358 |
| `mileage` | ≥ 0 | Negative mileage is not possible. | 369 | 0 |
| `tax` | ≥ 0 | Negative tax is not possible. | 378 | 0 |
| `mpg` | 5 to 150 | Lower bound 5 mpg is a conservative “sanity floor” below the least-efficient passenger car on FuelEconomy.gov’s list (Bugatti Mistral at 9 mpg combined), so we avoid removing valid low-efficiency cars while filtering implausible entries. [2] Upper bound 150 is a pragmatic cap to reduce leverage from extreme values and potential metric-mixing (e.g., MPGe-style values are defined as an energy-equivalent MPG for plug-in vehicles). [3] Reference point for high-efficiency non-EVs: Prius variants are ~50–56 mpg combined on FuelEconomy.gov. [4] | 49 | 221 |
| `engineSize` | 0.1 to 12.7 | Practical bounds: kei-class cars are capped at 660cc (0.66L), giving a grounded “small production car” reference point. [4] Very large historical production engines reach ~12.763L (Bugatti Type 41 / Royale). [5] Lower bound reduced to 0.1L as a conservative data-sanity floor (primarily to remove obvious errors) while avoiding unnecessary loss of potentially valid small-displacement entries. | 264 | 0 |
| `paintQuality%` → `paintQuality` | 0 to 100 | Percentage values must be between 0 and 100. | 0 | 367 |
| `previousOwners` | ≥ 0 | Negative owner counts are not possible. | 371 | 0 |
| `hasDamage` | - | Only 0 and NaN values in the data -> no thresholding | . | . |


[[1]: https://group.mercedes-benz.com/company/tradition/company-history/1885-1886.html [2]: https://www.fueleconomy.gov/feg/best-worst.shtml https://www.fueleconomy.gov/feg/PowerSearch.do?action=noform&baseModel=Prius&make=Toyota&path=1&srchtyp=ymm&year1=2020&year2=2020 [3]: https://www.epa.gov/greenvehicles/fuel-economy-and-ev-range-testing [4]: https://www.fueleconomy.gov/feg/PowerSearch.do?action=noform&baseModel=Prius&make=Toyota&path=1&srchtyp=ymm&year1=2020&year2=2020 [5]: https://www.motortrend.com/features/what-is-a-kei-car [6]: https://www.bugatti-trust.co.uk/bugatti-type-41/]

----

**Categorial Features**

We fix data entry noise (typos, truncations, inconsistent casing) in two stages:

**1) Deterministic canonicalization:**

We first apply **static mapping tables** (built from our EDA findings) that collapse variants into one canonical label

- normalize formatting: lowercase + trim whitespace  
- map known variants/typos → one canonical category  
  - example: `"AUDI"`, `"udi"`, `"Aud"` → `Audi`  
  - example: `"semi-aut"`, `"emi-auto"` → `Semi-Auto`

This step is **fully deterministic** and does not guess.

**2) Strict validity check:**

After canonicalization, we apply a **valid-category filter**:

- for high-cardinality `model`, we accept only **known valid model names** (our static canonical model vocabulary)
- any model token not in that vocabulary is considered unreliable and is set to **NaN**
- this prevents “garbage categories” (rare or malformed strings) from becoming real categories and harming generalization

Examples of what gets rejected (→ NaN):
- truncations like `"scirocc"` (intended: `scirocco`)
- incomplete single-letter tokens like `"a"`, `"q"`, `"x"` without enough context

**3) Special-rule buckets for ambiguous 1-letter tokens:**

Some one-letter model tokens are ambiguous and must not be guessed:

- if `brand == Audi` and `model == "a"` → set to `a_unknown`
- if `brand == Audi` and `model == "q"` → set to `q_unknown`
- if `brand == BMW` and `model == "x"` → set to `x_unknown`

If the brand is missing and the token is one of `{a, q, x}`, we do **not** guess and force it to **NaN**.

**4) Conservative fuzzy rescue (only for values that are still missing):**

Only after steps (1)–(3), we run a very strict “rescue step” to recover obvious typos:

- applied **only where the value is still NaN**
- candidate choices are restricted to **valid canonical categories** (never invent new labels)
- acceptance requires very high similarity (strict cutoff) and/or a unique prefix match  
  - example: `"pum"` → `puma` (unique prefix, safe)
  - example: `"sl clas"` → `sl class` (high similarity, safe)
- if no safe match exists, the value remains **NaN**

**5) Final handling of remaining missing categories:**

Any categorical value that is still missing after all steps remains **NaN** and is filled later by our **GroupImputer** (Section 2.3), using:
- mode within (brand, model) if possible  
- then mode within brand  
- then global mode

_____

**Leakage safety:**
All “data-driven vocabularies” used in the cleaner are learned inside `fit()` on the training fold only, so the pipeline remains leakage-safe under cross-validation:
- the validation fold is never used to decide which categories are “valid”
- the same cleaning logic is applied consistently in train/validation/test

*(References remain the numeric-threshold references above; the categorical process is internal and derived from EDA + controlled mappings.)*

